In [1]:
import pymongo
import json
import pandas as pd
from datetime import datetime, timedelta
from pandas import DataFrame
import matplotlib.pyplot as plt
from datetime import time
from tqdm import tqdm

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["studentlife"]

In [2]:
collection_charge = mydb["phonecharge"]
collection_lock = mydb["phonelock"]
collection_dark = mydb["dark"]

In [3]:
df_data = pd.DataFrame() 

In [4]:
phone_uid = ['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u07', 'u08', 'u09', 'u10', 'u12', 'u13', 'u14', 'u15', 'u16',
                'u17', 'u18', 'u19', 'u20', 'u22', 'u23', 'u24', 'u25', 'u27', 'u30', 'u31', 'u32', 'u33', 'u34', 'u35',
                'u36', 'u39', 'u41', 'u42', 'u43', 'u44', 'u45', 'u46', 'u47', 'u49', 'u50', 'u51', 'u52', 'u53', 'u54',
                'u56', 'u57', 'u58', 'u59']

In [5]:
morning_start_time = time(6, 0, 0)     # 6:00 AM
afternoon_start_time = time(12, 0, 0)  # 12:00 PM
evening_start_time = time(18, 0, 0)    # 6:00 PM
night_start_time = time(0, 0, 0)       # 12:00 AM (midnight)

In [6]:
def process_phone_data_for_uid(uid):
    result_dict = {}

    time_intervals = ["morning", "afternoon", "evening", "night"]

    for doc in collection_charge.find({"uid": uid}):
        if "timestamp" in doc and "end_timestamp" in doc:
            timestamp = int(doc["timestamp"])
            end_timestamp = int(doc["end_timestamp"])

            current_date = datetime.utcfromtimestamp(timestamp).strftime("%Y-%m-%d")
            current_time = datetime.utcfromtimestamp(timestamp).time()

            if morning_start_time <= current_time < afternoon_start_time:
                time_interval = "morning"
            elif afternoon_start_time <= current_time < evening_start_time:
                time_interval = "afternoon"
            elif evening_start_time <= current_time <= time(23, 59, 59):
                time_interval = "evening"
            else:
                time_interval = "night"

            duration = end_timestamp - timestamp

            key = f"{current_date}_{time_interval}"

            if key not in result_dict:
                result_dict[key] = {"Date": current_date}
            
            duration_column_name = f"{time_interval}_phonecharge_duration"
            frequency_column_name = f"{time_interval}_phonecharge_frequency"

            result_dict[key][duration_column_name] = result_dict[key].get(duration_column_name, 0) + duration
            result_dict[key][frequency_column_name] = result_dict[key].get(frequency_column_name, 0) + 1

    for doc in collection_lock.find({"uid": uid}):
        if "timestamp" in doc and "end_timestamp" in doc:
            timestamp = int(doc["timestamp"])
            end_timestamp = int(doc["end_timestamp"])

            current_date = datetime.utcfromtimestamp(timestamp).strftime("%Y-%m-%d")
            current_time = datetime.utcfromtimestamp(timestamp).time()

            if morning_start_time <= current_time < afternoon_start_time:
                time_interval = "morning"
            elif afternoon_start_time <= current_time < evening_start_time:
                time_interval = "afternoon"
            elif evening_start_time <= current_time <= time(23, 59, 59):
                time_interval = "evening"
            else:
                time_interval = "night"

            duration = end_timestamp - timestamp
            key = f"{current_date}_{time_interval}"

            if key not in result_dict:
                result_dict[key] = {"Date": current_date}

            duration_column_name = f"{time_interval}_phonelock_duration"
            frequency_column_name = f"{time_interval}_phonelock_frequency"

            result_dict[key][duration_column_name] = result_dict[key].get(duration_column_name, 0) + duration
            result_dict[key][frequency_column_name] = result_dict[key].get(frequency_column_name, 0) + 1

    for doc in collection_dark.find({"uid": uid}):
        if "timestamp" in doc and "end_timestamp" in doc:
            timestamp = int(doc["timestamp"])
            end_timestamp = int(doc["end_timestamp"])

            current_date = datetime.utcfromtimestamp(timestamp).strftime("%Y-%m-%d")
            current_time = datetime.utcfromtimestamp(timestamp).time()

            if morning_start_time <= current_time < afternoon_start_time:
                time_interval = "morning"
            elif afternoon_start_time <= current_time < evening_start_time:
                time_interval = "afternoon"
            elif evening_start_time <= current_time <= time(23, 59, 59):
                time_interval = "evening"
            else:
                time_interval = "night"

            duration = end_timestamp - timestamp
            key = f"{current_date}_{time_interval}"

            if key not in result_dict:
                result_dict[key] = {"Date": current_date}

            duration_column_name = f"{time_interval}_dark_duration"

            result_dict[key][duration_column_name] = result_dict[key].get(duration_column_name, 0) + duration

    result_list = list(result_dict.values())

    df = pd.DataFrame(result_list)
    df.fillna(0, inplace=True)
    df['uid'] = uid

    return df

In [7]:
for uid in tqdm(phone_uid):
    print(uid)
    df = process_phone_data_for_uid(str(uid))
    print(df.head(3))
    frames = [df_data, df]
    print(frames)
    df_data = pd.concat(frames, ignore_index=True)

 14%|██████▎                                     | 7/49 [00:00<00:00, 61.30it/s]

u00
         Date  night_phonecharge_duration  night_phonecharge_frequency  \
0  2013-03-27                      5921.0                          1.0   
1  2013-03-27                         0.0                          0.0   
2  2013-03-28                         0.0                          0.0   

   night_phonelock_duration  night_phonelock_frequency  \
0                    5815.0                        1.0   
1                       0.0                        0.0   
2                       0.0                        0.0   

   morning_phonecharge_duration  morning_phonecharge_frequency  \
0                           0.0                            0.0   
1                       16650.0                            1.0   
2                           0.0                            0.0   

   morning_phonelock_duration  morning_phonelock_frequency  \
0                         0.0                          0.0   
1                     16368.0                          1.0   
2              

 43%|██████████████████▍                        | 21/49 [00:00<00:00, 60.05it/s]

         Date  night_phonecharge_duration  night_phonecharge_frequency  \
0  2013-03-27                     31869.0                          1.0   
1  2013-03-28                     31673.0                          1.0   
2  2013-03-29                     30742.0                          1.0   

   night_phonelock_duration  night_phonelock_frequency  night_dark_duration  \
0                   30706.0                        1.0              30703.0   
1                   42136.0                        2.0              45813.0   
2                   30709.0                        1.0              30737.0   

   morning_phonecharge_duration  morning_phonecharge_frequency  \
0                           0.0                            0.0   
1                           0.0                            0.0   
2                           0.0                            0.0   

   morning_phonelock_duration  morning_phonelock_frequency  ...  \
0                         0.0                         

 71%|██████████████████████████████▋            | 35/49 [00:00<00:00, 59.82it/s]

         Date  night_phonecharge_duration  night_phonecharge_frequency  \
0  2013-03-28                     12158.0                          1.0   
1  2013-03-28                         0.0                          0.0   
2  2013-03-29                     13417.0                          1.0   

   night_phonelock_duration  night_phonelock_frequency  \
0                   11139.0                        1.0   
1                       0.0                        0.0   
2                    6871.0                        1.0   

   morning_phonecharge_duration  morning_phonecharge_frequency  \
0                           0.0                            0.0   
1                       10638.0                            1.0   
2                           0.0                            0.0   

   morning_phonelock_duration  morning_phonelock_frequency  \
0                         0.0                          0.0   
1                      3778.0                          1.0   
2                  

100%|███████████████████████████████████████████| 49/49 [00:00<00:00, 60.80it/s]

[            Date  night_phonecharge_duration  night_phonecharge_frequency  \
0     2013-03-27                      5921.0                          1.0   
1     2013-03-27                         0.0                          0.0   
2     2013-03-28                         0.0                          0.0   
3     2013-03-29                      7074.0                          1.0   
4     2013-03-29                         0.0                          0.0   
...          ...                         ...                          ...   
5537  2013-04-15                         0.0                          0.0   
5538  2013-04-30                         0.0                          0.0   
5539  2013-04-30                         0.0                          0.0   
5540  2013-05-05                         0.0                          0.0   
5541  2013-05-05                         0.0                          0.0   

      night_phonelock_duration  night_phonelock_frequency  \
0            

In [8]:
len(df_data)

7558

In [9]:
desired_order = ['Date', 'uid', 
                 'morning_phonecharge_duration', 'morning_phonecharge_frequency', 'morning_phonelock_duration', 'morning_phonelock_frequency', 'morning_dark_duration',
                 'afternoon_phonecharge_duration', 'afternoon_phonecharge_frequency', 'afternoon_phonelock_duration', 'afternoon_phonelock_frequency', 'afternoon_dark_duration',
                 'evening_phonecharge_duration', 'evening_phonecharge_frequency', 'evening_phonelock_duration', 'evening_phonelock_frequency', 'evening_dark_duration',
                 'night_phonecharge_duration', 'night_phonecharge_frequency', 'night_phonelock_duration', 'night_phonelock_frequency', 'night_dark_duration']

df_data = df_data[desired_order]

In [10]:
df_data.to_csv('raw_phonelog_features_studentlife.csv')

In [11]:
df_data.isnull().sum()

Date                                 0
uid                                  0
morning_phonecharge_duration       406
morning_phonecharge_frequency      406
morning_phonelock_duration          12
morning_phonelock_frequency         12
morning_dark_duration               99
afternoon_phonecharge_duration       0
afternoon_phonecharge_frequency      0
afternoon_phonelock_duration         0
afternoon_phonelock_frequency        0
afternoon_dark_duration              0
evening_phonecharge_duration        12
evening_phonecharge_frequency       12
evening_phonelock_duration           0
evening_phonelock_frequency          0
evening_dark_duration                0
night_phonecharge_duration          12
night_phonecharge_frequency         12
night_phonelock_duration             0
night_phonelock_frequency            0
night_dark_duration                  0
dtype: int64